In [2]:
DATASET_PATH = "/Users/mac_user/PlantVillage-Dataset/raw"


In [10]:
#This code is setting up data preprocessing for training and validating a deep learning model using images.  
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 130333 images belonging to 3 classes.
Found 32583 images belonging to 3 classes.


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base layers

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(train_gen.num_classes, activation='softmax')  # Adjusts automatically to class count
])


In [5]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [6]:
EPOCHS = 5  # Start small; increase if needed

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)


Epoch 1/5


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4073/4073 ━━━━━━━━━━━━━━━━━━━━ 2603s 638ms/step - accuracy: 0.9546 - loss: 0.1252 - val_accuracy: 0.8910 - val_loss: 0.3937
Epoch 2/5
4073/4073 ━━━━━━━━━━━━━━━━━━━━ 9221s 2s/step - accuracy: 0.9951 - loss: 0.0175 - val_accuracy: 0.8903 - val_loss: 0.4093
Epoch 3/5
4073/4073 ━━━━━━━━━━━━━━━━━━━━ 16885s 4s/step - accuracy: 0.9951 - loss: 0.0149 - val_accuracy: 0.8936 - val_loss: 0.3871
Epoch 4/5
4073/4073 ━━━━━━━━━━━━━━━━━━━━ 2337s 574ms/step - accuracy: 0.9962 - loss: 0.0122 - val_accuracy: 0.8953 - val_loss: 0.4160
Epoch 5/5
4073/4073 ━━━━━━━━━━━━━━━━━━━━ 2484s 610ms/step - accuracy: 0.9969 - loss: 0.0098 - val_accuracy: 0.8946 - val_loss: 0.4154


In [8]:
model.save("CropSURE_model.keras")

In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

# Load the model
model = load_model("CropSURE_model.keras")  # Replace with your saved model path

# Load and preprocess the test image
img_path = "/Users/mac_user/test_leaf.png"  # Replace with the correct path to your image
img = image.load_img(img_path, target_size=(224, 224))  # Resize image to match model input
img_array = image.img_to_array(img) / 255.0  # Convert to array and normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict using the trained model
prediction = model.predict(img_array)

# Get class labels from training data (reuse train_gen)
class_indices = {'Tomato___Early_blight': 0, 'Tomato___Late_blight': 1}  # Replace with actual mapping
labels = {v: k for k, v in class_indices.items()}

# Print the predicted class
predicted_class = labels[np.argmax(prediction)]
print(f"Predicted Disease: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step
Predicted Disease: Tomato___Early_blight
